In [ ]:
# Model architecture inspired by Z. Hameed and B. Garcia-Zapirain, "Sentiment classification using a single-layered BiLSTM model", IEEE Access, vol. 8, pp. 73992-74001, 2020.
!pip install tensorflow==1.15
!pip install tensorflow_hub>=0.6.0
!pip3 install tensorflow_text==1.15

     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 3.8 MB 40.3 MB/s 
     |████████████████████████████████| 503 kB 41.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2923a3c11c2cabfa36ec41ea7d6d6b0ebedd74c37f78541fa02850339747d77e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Input, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate

from tensorflow.keras.layers import Lambda

def process_text(document):
     
    # Remove extra white space from text
    document = re.sub(r'\s+', ' ', document, flags=re.I)
         
    # Remove all the special characters from text
    document = re.sub(r'\W', ' ', str(document))
 
    # Word tokenization       
    tokens = document.split()
        
    tokens = [word for word in tokens if len(word) > 2]
 
    return tokens

In [ ]:
df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

,Emotion,Statement
0,joy,"Thank you , Steven . I accept your advice ."
1,surprise,"Oh my God, I can't believe I have two-two chil..."
2,neutral,"Look, it's not that big of a deal, so Monica a..."
3,anger,And I wanna know why?!!
4,joy,"And, ah, you know, your fooling around with her."


In [ ]:
from tqdm import tqdm 

df['preprocessedStatement'] = df.Statement.apply(process_text)
display(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Emotion,Statement,preprocessedStatement
0,joy,"Thank you , Steven . I accept your advice .","[thank, you, steven, accept, your, advice]"
1,surprise,"Oh my God, I can't believe I have two-two chil...","[god, can, believe, have, two, two, children]"
2,neutral,"Look, it's not that big of a deal, so Monica a...","[look, not, that, big, deal, monica, and, chan..."
3,anger,And I wanna know why?!!,"[and, wanna, know, why]"
4,joy,"And, ah, you know, your fooling around with her.","[and, you, know, your, fooling, around, with, ..."


In [ ]:
max_length = df.preprocessedStatement.apply(lambda x: len(x)).max()

t = Tokenizer()
t.fit_on_texts(df['preprocessedStatement'] )
vocab_size = len(t.word_index) + 1

In [ ]:
new_X = []
for seq in df['preprocessedStatement']:
    new_seq = []
    for i in range(max_length):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# Encode labels in column 'Emotion'. 
df['Emotion'] = le.fit_transform(df['Emotion']) 
y = df.pop('Emotion')
y_new = tf.keras.utils.to_categorical(y, num_classes=7)
print(y_new)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
new_X_df = pd.DataFrame(new_X)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(new_X_df, y_new, test_size=0.04, stratify=y_new)

In [ ]:
#for meld-dd
x_val, y_val = x_train[:174], y_train[:174] 
x_train, y_train = x_train[174:], y_train[174:]
x_val, y_val = x_val[:128], y_val[:128] 
x_test, y_test = x_test[:1216], y_test[:1216]

# #for isear
# x_val, y_val = x_train[:143], y_train[:143] 
# x_train, y_train = x_train[143:], y_train[143:]
# x_val, y_val = x_val[:128], y_val[:128] 
# x_test, y_test = x_test[:288], y_test[:288]

In [ ]:
batch_size = 64 #32 for Isear, 64 for Meld-dd

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import backend as K

sess = tf.Session()
K.set_session(sess)

elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_length])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)
input_layer = Input(shape=(max_length, ), batch_size = batch_size, dtype=tf.string) 
embedding = Lambda(ElmoEmbedding, output_shape=(max_length, 1024))(input_layer)
x = Bidirectional(LSTM(64, return_sequences=True))(embedding) 
x_a = GlobalMaxPool1D()(x)
x_b = GlobalAveragePooling1D()(x)
x = concatenate([x_a,x_b])
x = Dense(128, activation="relu")(x) 
x = Dense(7, activation='softmax')(x)
model_elmo = Model(inputs=input_layer, outputs=x)
model_elmo.compile(loss=focal_loss(alpha=1), optimizer='adam', metrics=['accuracy']) #Alternative: tf.keras.metrics.Recall() as metric 
model_elmo.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(64, 151)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (64, 151, 1024)      0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (64, 151, 128)       557568      lambda[0][0]                     
__________________________________________________________________________________________________
global_max_pooling1d (GlobalMax (64, 128)            0           bidirectional[0][0]              
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_elmo,show_shapes= True)

In [ ]:
model_elmo.fit(x_train, y_train, epochs = 10, callbacks=[callback], validation_data=(x_val, y_val)) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 29696 samples, validate on 128 samples
Epoch 1/10
29696/29696 [==============================] - 1213s 41ms/sample - loss: 0.7069 - acc: 0.5941 - val_loss: 0.6630 - val_acc: 0.5703
Epoch 2/10
29696/29696 [==============================] - 1204s 41ms/sample - loss: 0.5979 - acc: 0.6489 - val_loss: 0.6247 - val_acc: 0.6328
Epoch 3/10
29696/29696 [==============================] - 1211s 41ms/sample - loss: 0.5405 - acc: 0.6776 - val_loss: 0.6354 - val_acc: 0.6406
Epoch 4/10
29696/29696 [==============================] - 1209s 41ms/sample - loss: 0.4857 - acc: 0.7033 - val_loss: 0.6620 - val_acc: 0.6094
Epoch 5/10
29696/29696 [==============================] - 1207s 41ms/sample - loss: 0.4217 - acc: 0.7328 - val_loss: 0.6433 - val_acc: 0.6328
Epoch 6/10
29696/29696 [==============================] - 1213s 41ms/sample - loss: 0.3554 - acc: 0.7639 - val_loss: 0.6649 - val_acc: 0.6484
Epoch 7/10
29696/29696 [==============================] - 1226s 41ms/sample - loss: 0.2906 - acc: 0.

In [ ]:
y_pred = model_elmo.predict(x_test)

In [ ]:
y_pred_clean = np.argmax(y_pred, 1)
y_test_clean = np.argmax(y_test, 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_clean, y_pred_clean))

              precision    recall  f1-score   support

           0       0.48      0.40      0.44       103
           1       0.32      0.28      0.30        29
           2       0.67      0.20      0.31        20
           3       0.76      0.84      0.80       596
           4       0.50      0.47      0.48       249
           5       0.42      0.44      0.43        85
           6       0.58      0.49      0.53       134

    accuracy                           0.64      1216
   macro avg       0.53      0.45      0.47      1216
weighted avg       0.63      0.64      0.63      1216

